In [2]:
library(tidyverse)
abalone_data <- read_csv("abalone.data", col_names = FALSE)
abalone_data <- rename(abalone_data, 
                       sex = X1,
                       length_mm = X2,
                       diameter_mm = X3,
                       height_mm = X4,
                       whole_weight_g = X5,
                       shucked_weight_g = X6,
                       viscera_weight_g = X7,
                       shell_weight_g = X8,
                       rings = X9)
abalone_data <- mutate(abalone_data, age_yrs = rings + 1.5)
head(abalone_data)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.2     ✔ purrr   1.0.1
✔ tibble  3.2.1     ✔ dplyr   1.1.1
✔ tidyr   1.3.0     ✔ stringr 1.5.0
✔ readr   2.1.3     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Rows: 4177 Columns: 9
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): X1
dbl (8): X2, X3, X4, X5, X6, X7, X8, X9

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


sex,length_mm,diameter_mm,height_mm,whole_weight_g,shucked_weight_g,viscera_weight_g,shell_weight_g,rings,age_yrs
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15,16.5
M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7,8.5
F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9,10.5
M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10,11.5
I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7,8.5
I,0.425,0.300,0.095,0.3515,0.1410,0.0775,0.120,8,9.5


In [3]:
abalone_data <- select(abalone_data, sex, length_mm, whole_weight_g, age_yrs)
head(abalone_data)

sex,length_mm,whole_weight_g,age_yrs
<chr>,<dbl>,<dbl>,<dbl>
M,0.455,0.5140,16.5
M,0.350,0.2255,8.5
F,0.530,0.6770,10.5
M,0.440,0.5160,11.5
I,0.330,0.2050,8.5
I,0.425,0.3515,9.5


### Methods
For our analysis we will only be using the length and whole weight of the abalone to predict it's age through use of regression. The length of an abalone is used in practice to determine maturity of the marine snail and the whole weight can be used to show growth, both of which are indicative of age. Using the length and whole weight of an abalone to predict age can be visualized through a scatter plot. 

### Splitting into Training Data

since age is a categorical variable, we can split the data into training and test data using 'initial_split()'


In [4]:
set.seed(1)

abalone_data <- abalone_data |>
  mutate(sex = as_factor(sex))  |>
  mutate(sex = fct_recode(sex, "Male" = "M", "Female" = "F", "Infant" = "I"))
head(abalone_data)
glimpse(abalone_data)

sex,length_mm,whole_weight_g,age_yrs
<fct>,<dbl>,<dbl>,<dbl>
Male,0.455,0.5140,16.5
Male,0.350,0.2255,8.5
Female,0.530,0.6770,10.5
Male,0.440,0.5160,11.5
Infant,0.330,0.2050,8.5
Infant,0.425,0.3515,9.5


Rows: 4,177
Columns: 4
$ sex            <fct> Male, Male, Female, Male, Infant, Infant, Female, Femal…
$ length_mm      <dbl> 0.455, 0.350, 0.530, 0.440, 0.330, 0.425, 0.530, 0.545,…
$ whole_weight_g <dbl> 0.5140, 0.2255, 0.6770, 0.5160, 0.2050, 0.3515, 0.7775,…
$ age_yrs        <dbl> 16.5, 8.5, 10.5, 11.5, 8.5, 9.5, 21.5, 17.5, 10.5, 20.5…


In [5]:
library(tidymodels)

abalone_split <- initial_split(abalone_data, prop = 0.75, strata = sex)
abalone_train <- training(abalone_split)
abalone_test <- testing(abalone_split)

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.2     ✔ rsample      1.1.1
✔ dials        1.1.0     ✔ tune         1.0.1
✔ infer        1.0.4     ✔ workflows    1.1.2
✔ modeldata    1.0.1     ✔ workflowsets 1.0.0
✔ parsnip      1.0.3     ✔ yardstick    1.1.0
✔ recipes      1.0.4     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks readr::spec()
✖ recipes::step()   masks stats::step()
• Dig deeper into tidy modeling with R at https://www.tmwr.org



In [6]:
glimpse(abalone_train)

Rows: 3,132
Columns: 4
$ sex            <fct> Female, Female, Female, Female, Female, Female, Female,…
$ length_mm      <dbl> 0.545, 0.525, 0.535, 0.470, 0.440, 0.615, 0.580, 0.680,…
$ whole_weight_g <dbl> 0.7680, 0.6065, 0.6845, 0.4755, 0.4510, 1.1615, 0.9955,…
$ age_yrs        <dbl> 17.5, 15.5, 11.5, 11.5, 11.5, 11.5, 12.5, 16.5, 20.5, 1…


In [7]:
glimpse(abalone_test)

Rows: 1,045
Columns: 4
$ sex            <fct> Female, Infant, Female, Female, Male, Male, Female, Fem…
$ length_mm      <dbl> 0.530, 0.425, 0.530, 0.550, 0.365, 0.450, 0.565, 0.550,…
$ whole_weight_g <dbl> 0.6770, 0.3515, 0.7775, 0.8945, 0.2555, 0.3810, 0.9395,…
$ age_yrs        <dbl> 10.5, 9.5, 21.5, 20.5, 8.5, 10.5, 13.5, 10.5, 12.5, 14.…


We can check to see how representative the training data is by comparing the proportion of each unique observation in the 'sex' variable across the training data 'abalone_train' and the original data 'abalone_data'. 

In [8]:
#Checking for proportions of male, female, and infant abalone's in the original data
abalone_proportions <- abalone_data |>
                      group_by(sex) |>
                      summarize(n = n()) |>
                      mutate(percent = 100*n/nrow(abalone_data))

abalone_proportions

sex,n,percent
<fct>,<int>,<dbl>
Male,1528,36.58128
Female,1307,31.29040
Infant,1342,32.12832


In [9]:
#Checking for proportions of male, female, and infant abalone's in the training data
abalone_train_proportions <- abalone_train |>
                      group_by(sex) |>
                      summarize(n = n()) |>
                      mutate(percent = 100*n/nrow(abalone_train))

abalone_train_proportions

sex,n,percent
<fct>,<int>,<dbl>
Male,1146,36.59004
Female,980,31.28991
Infant,1006,32.12005


Both 'abalone_data' and 'abalone_train' have similar proportions within the classifier 'sex' indicating that proportions were preserved after the split.

Before using the training data, we need to make sure our predictor variables are standardized within the training data set.

In [10]:
abalone_recipe <- recipe(sex ~ length_mm + whole_weight_g + age_yrs, data = abalone_train) |>
  step_scale(all_predictors()) |>
  step_center(all_predictors())